# HS Spring 2020 Data Analysis

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib
import matplotlib.pyplot as plt

import re
import argparse
import logging

import os
# import glob
import sys
from pathlib import Path


from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist

import gamework

In [3]:
#help(gamework.MatchData)

root_dir = 'C:\\Users\\Main\\Documents\\GitHub\\Sabber_Work_2019F\\thesis-output\\ZLvsCL_1k0_Compiled'

#so_pizza = os.path.join(root_dir, '')

In [4]:
md = gamework.MatchupData('aggro', 'aggro', root_dir)

In [5]:
md.get_matchups()

Roffle-Roffle.csv
Roffle-Viper.csv
Roffle-Pizza.csv
Roffle-Solegit.csv
Roffle-Wabeka.csv
Viper-Roffle.csv
Viper-Viper.csv
Viper-Pizza.csv
Viper-Solegit.csv
Viper-Wabeka.csv
Pizza-Roffle.csv
Pizza-Viper.csv
Pizza-Pizza.csv
Pizza-Solegit.csv
Pizza-Wabeka.csv
Solegit-Roffle.csv
Solegit-Viper.csv
Solegit-Pizza.csv
Solegit-Solegit.csv
Solegit-Wabeka.csv
Wabeka-Roffle.csv
Wabeka-Viper.csv
Wabeka-Pizza.csv
Wabeka-Solegit.csv
Wabeka-Wabeka.csv
Total 25 matchups


In [6]:
all_folders = md.get_folders()
all_folders

['C:\\Users\\Main\\Documents\\GitHub\\Sabber_Work_2019F\\thesis-output\\ZLvsCL_1k0_Compiled\\Roffle-Roffle.csv',
 'C:\\Users\\Main\\Documents\\GitHub\\Sabber_Work_2019F\\thesis-output\\ZLvsCL_1k0_Compiled\\Roffle-Viper.csv',
 'C:\\Users\\Main\\Documents\\GitHub\\Sabber_Work_2019F\\thesis-output\\ZLvsCL_1k0_Compiled\\Roffle-Pizza.csv',
 'C:\\Users\\Main\\Documents\\GitHub\\Sabber_Work_2019F\\thesis-output\\ZLvsCL_1k0_Compiled\\Roffle-Solegit.csv',
 'C:\\Users\\Main\\Documents\\GitHub\\Sabber_Work_2019F\\thesis-output\\ZLvsCL_1k0_Compiled\\Roffle-Wabeka.csv',
 'C:\\Users\\Main\\Documents\\GitHub\\Sabber_Work_2019F\\thesis-output\\ZLvsCL_1k0_Compiled\\Viper-Roffle.csv',
 'C:\\Users\\Main\\Documents\\GitHub\\Sabber_Work_2019F\\thesis-output\\ZLvsCL_1k0_Compiled\\Viper-Viper.csv',
 'C:\\Users\\Main\\Documents\\GitHub\\Sabber_Work_2019F\\thesis-output\\ZLvsCL_1k0_Compiled\\Viper-Pizza.csv',
 'C:\\Users\\Main\\Documents\\GitHub\\Sabber_Work_2019F\\thesis-output\\ZLvsCL_1k0_Compiled\\Viper-Sol

In [9]:
match1 = gamework.MatchData(all_folders[0])
match1_summary = match1.get_summary()

(4134, 23)


In [12]:
match1_summary['EoT-Standard-Dev'][1]

TURN_NO                                     0.000000
P1_HEALTH                                   0.000000
P2_HEALTH                                   1.773233
AMOUNTHEALEDTHISTURN                        0.000000
NUMATTACKSTHISTURN                          0.000000
NUMCARDSDRAWNTHISTURN                       0.000000
NUMCARDSPLAYEDTHISTURN                      0.264945
NUMCARDSTODRAW                              0.000000
NUMELEMENTALSPLAYEDLASTTURN                 0.000000
NUMELEMENTALSPLAYEDTHISTURN                 0.000000
NUMFRIENDLYMINIONSTHATATTACKEDTHISTURN      0.000000
NUMFRIENDLYMINIONSTHATDIEDTHISTURN          0.000000
NUMMINIONSPLAYEDTHISTURN                    0.475456
NUMMINIONSPLAYERKILLEDTHISTURN              0.000000
NUMOPTIONSPLAYEDTHISTURN                    0.264945
NUMSECRETSPLAYEDTHISGAME                    0.000000
NUMSPELLSPLAYEDTHISGAME                     0.443308
NUMTIMESHEROPOWERUSEDTHISGAME               0.000000
REMAININGMANA                               0.